In [50]:
# server_ip = "cloudflare-quic.com/"
nginx = {
    "name": "nginx",
    "ip": "3.69.173.4",
}

h2o = {
    "name": "h2o",
    "ip": "18.184.62.114",
}

caddy = {
    "name": "caddy",
    "ip": "3.69.24.93",
}

iperf_port = "6969"

servers = (nginx, h2o, caddy)

## Bandwidth and latency test

### Latency

In [20]:
for server in servers:
    print("")
    print(f"==================== PING {server['name']} ==========================")
    !ping -c 5 {server["ip"]}


==================== PING nginx ==========================
PING 3.69.173.4 (3.69.173.4) 56(84) bytes of data.
64 bytes from 3.69.173.4: icmp_seq=1 ttl=63 time=0.482 ms
64 bytes from 3.69.173.4: icmp_seq=2 ttl=63 time=0.394 ms
64 bytes from 3.69.173.4: icmp_seq=3 ttl=63 time=0.475 ms
64 bytes from 3.69.173.4: icmp_seq=4 ttl=63 time=0.431 ms
64 bytes from 3.69.173.4: icmp_seq=5 ttl=63 time=0.377 ms

--- 3.69.173.4 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4099ms
rtt min/avg/max/mdev = 0.377/0.431/0.482/0.042 ms

==================== PING h2o ==========================
PING 18.184.62.114 (18.184.62.114) 56(84) bytes of data.
64 bytes from 18.184.62.114: icmp_seq=1 ttl=63 time=0.619 ms
64 bytes from 18.184.62.114: icmp_seq=2 ttl=63 time=0.547 ms
64 bytes from 18.184.62.114: icmp_seq=3 ttl=63 time=0.564 ms
64 bytes from 18.184.62.114: icmp_seq=4 ttl=63 time=0.553 ms
64 bytes from 18.184.62.114: icmp_seq=5 ttl=63 time=0.539 ms

--- 18.184.62.114 ping statis

### Bandwidth - TCP & UDP

In [21]:
# TODO: Make sure iperf is running on all servers.
!iperf -c {nginx["ip"]} -p {iperf_port}
!iperf -u -c {nginx["ip"]} -p {iperf_port} -b 0

Connecting to host 3.69.173.4, port 6969
[  5] local 172.31.13.42 port 36580 connected to 3.69.173.4 port 6969
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-1.00   sec   360 MBytes  3.02 Gbits/sec  328    532 KBytes       
[  5]   1.00-2.00   sec   378 MBytes  3.17 Gbits/sec  237    301 KBytes       
[  5]   2.00-3.00   sec   398 MBytes  3.33 Gbits/sec  211    433 KBytes       
[  5]   3.00-4.00   sec   439 MBytes  3.68 Gbits/sec  389    542 KBytes       
[  5]   4.00-5.00   sec   488 MBytes  4.09 Gbits/sec  343    400 KBytes       
[  5]   5.00-6.00   sec   460 MBytes  3.86 Gbits/sec  287    580 KBytes       
[  5]   6.00-7.00   sec   412 MBytes  3.46 Gbits/sec  366    673 KBytes       
[  5]   7.00-8.00   sec   512 MBytes  4.30 Gbits/sec  351    550 KBytes       
[  5]   8.00-9.00   sec   511 MBytes  4.29 Gbits/sec  590    375 KBytes       
[  5]   9.00-10.00  sec   455 MBytes  3.82 Gbits/sec  360    481 KBytes       
- - - - - - - - - - - - - - - - - 

### Measuring QUIC performance

#### h2 load output parsing

In [5]:
def get_throughput(res):
    fin_line = [l for l in res if l.startswith("finished in ")][0]
    return float(fin_line.split(" ")[-1][:-4])

def get_rps(res):
    fin_line = [l for l in res if l.startswith("finished in ")][0]
    return float(fin_line.split(" ")[3])

def get_success_pct(res):
    req_line = [l for l in res if l.startswith("requests:")][0]
    items = req_line.split(" ")
    return float(items[7])/float(items[1])

#### Bulk Throughput

Get throughput measurment from the client.

In [29]:
for server in servers:
    for protocol in ("h2", "h3",):
        print(f"bulk throughput test {protocol} - {server['name']}")
        # Request the file first, just to warm up caches.
        _ = !h2load -n1 -c1 --npn-list={protocol} https://{server['ip']}/1g.txt
        for i in range(2):
            res = !h2load -n1 -c1 --npn-list={protocol} https://{server['ip']}/1g.txt
            print(f"{get_throughput(res)} MB/s (succ: {get_success_pct(res)})")

bulk throughput test h2 - nginx
415.24 MB/s (succ: 1.0)
437.79 MB/s (succ: 1.0)
bulk throughput test h3 - nginx
113.17 MB/s (succ: 1.0)
118.42 MB/s (succ: 1.0)
bulk throughput test h2 - h2o
504.9 MB/s (succ: 1.0)
480.44 MB/s (succ: 1.0)
bulk throughput test h3 - h2o
268.42 MB/s (succ: 1.0)
258.71 MB/s (succ: 1.0)


In [49]:
for server in servers:
    for protocol in ("h2", "h3"):
        print(f"rps test {protocol} - {server['name']}")
        for i in range(2):
            res = !h2load -n2000000 -t $(nproc) -c 50 -m 100 --npn-list={protocol} https://{server['ip']}/index.html
            print(f"{get_rps(res)} req/s (succ: {get_success_pct(res):.3f})")

rps test h2 - nginx
49906.62 req/s (succ: 1.000)
60974.8 req/s (succ: 1.000)
rps test h3 - nginx
100915.54 req/s (succ: 1.000)
99119.85 req/s (succ: 1.000)
rps test h2 - h2o
76161.61 req/s (succ: 1.000)
74966.83 req/s (succ: 1.000)
rps test h3 - h2o
67102.15 req/s (succ: 1.000)
67468.49 req/s (succ: 1.000)


In [65]:
!h2load -v -n1 -c1 --npn-list=h2 https://{caddy['ip']}/1g.txt

starting benchmark...
spawning thread #0: 1 total client(s). 1 total requests
^C
